In [1]:
import pickle

In [2]:
filename = "documents.pkl"
filepath = ""
fileuri = filepath+filename

In [3]:
with open(fileuri, 'rb') as documents: 
    docs = pickle.load(documents)

In [4]:
from llama_index.llms import HuggingFaceLLM
from llama_index.text_splitter import SentenceSplitter
from llama_index import VectorStoreIndex, ServiceContext

text_splitter = SentenceSplitter(
    chunk_size=200,
    chunk_overlap=20,
)

generate_kwargs={"temperature": 0.7, 
                 "do_sample": True, 
                 "top_k": 50, 
                 "top_p": 0.70
                }

llm = HuggingFaceLLM(
    context_window=	2048,
    max_new_tokens=256,
    generate_kwargs= generate_kwargs,
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    tokenizer_kwargs={"max_length": 2048},
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16}
)

service_context = ServiceContext.from_defaults(llm = llm, 
                                               text_splitter = text_splitter, 
                                               embed_model="local")

index = VectorStoreIndex.from_documents(
    docs, service_context=service_context
)



config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
index

In [6]:
import pymilvus
from milvus import default_server
from llama_index.vector_stores import MilvusVectorStore
from pymilvus import connections, Collection, utility

# Start Milvus Vector DB
default_server.stop()
default_server.set_base_dir('milvus-data')
default_server.start()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
import milvus
from llama_index.vector_stores import MilvusVectorStore
from llama_index.storage.storage_context import StorageContext

# vector_store = MilvusVectorStore(dim=2048, overwrite=True)

# vector_store = MilvusVectorStore(dim=2048, overwrite=True)
vector_store = MilvusVectorStore(dim=384, overwrite=True)


In [8]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [9]:
index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context, service_context=service_context
)

In [10]:
query_engine = index.as_query_engine()
res = query_engine.query("What is LlamaIndex?")
print("Res:", res)

Res: 
LlamaIndex is a data framework for LLM-based applications to ingest, structure, and access private or domain-specific data. It’s available in Python (these docs) and Typescript. It’s for beginners, advanced users, and everyone in between.


LlamaIndex is a data framework for LLM-based applications to ingest, structure, and access private or domain-specific data. It’s available in Python (these docs) and Typescript. LlamaIndex is designed for beginners, advanced users, and everyone in between. Its high-level API allows for easy ingestion and querying of data in 5 lines of code. Its lower-level APIs allow for customization and extension to fit complex applications.

In [11]:
docs[0]

Document(id_='398cc74a-580d-41c2-a85f-0b980e4ff0ef', embedding=None, metadata={'url': 'https://gpt-index.readthedocs.io/en/latest'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='767b9945a76c081d709c631a4c4bd5ea20b3b2ca30c6b52133fa78638aab3be3', text='LlamaIndex is a data framework for LLM-based applications to ingest, structure, and access private or domain-specific data. It’s available in Python (these docs) and Typescript.\n🚀 Why LlamaIndex?\uf0c1\nLLMs offer a natural language interface between humans and data. Widely available models come pre-trained on huge amounts of publicly available data like Wikipedia, mailing lists, textbooks, source code and more.\nHowever, while LLMs are trained on a great deal of data, they are not trained on your data, which may be private or specific to the problem you’re trying to solve. It’s behind APIs, in SQL databases, or trapped in PDFs and slide decks.\nYou may choose to fine-tune a LLM with your data, b

In [12]:
connection = connections.connect("default", host="localhost", port="19530")


In [13]:
utility.list_collections()

['llamalection', 'book']

In [14]:
collection = Collection("llamalection")
collection.schema

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': True}

In [15]:
collection.num_entities

68

In [20]:
res = collection.query(
  expr = "",
  offset = 60,
  limit = 10, 
  output_fields = ['id', 'embedding']
)

In [30]:
res[0]

{'id': 'd78b8eaa-8b81-4c8d-b21d-1a7fb39ac2ca',
 'embedding': [-0.06251519,
  -0.013404677,
  -0.022419333,
  -0.0057119303,
  -0.0048313807,
  0.018648615,
  0.010658559,
  -0.0039329743,
  0.008745981,
  -0.025675053,
  -0.013916663,
  -0.030394591,
  0.04816568,
  0.0051902444,
  -0.016413933,
  0.011856366,
  0.010102913,
  0.009473617,
  0.028741565,
  0.055245023,
  0.036948513,
  0.013249658,
  -0.057163365,
  0.0006234221,
  0.012859257,
  -0.003343579,
  -0.025322476,
  -0.015072759,
  -0.02962951,
  -0.20529802,
  -0.024722375,
  -0.0052649616,
  0.009839829,
  0.0047894204,
  0.007153571,
  0.014824283,
  0.015458113,
  -0.0195616,
  -0.027394686,
  0.006797099,
  -0.018606357,
  -0.009030563,
  0.03189251,
  0.01812224,
  0.034556318,
  -0.021895714,
  -0.029221345,
  -0.030029615,
  -0.062034365,
  -0.013681108,
  -0.03382084,
  0.0060293297,
  0.002489617,
  0.031486444,
  0.012832193,
  0.022034839,
  0.049162697,
  0.05765955,
  0.02551283,
  0.00721585,
  0.012845799,
 

In [29]:
len(res[0]['embedding'])

384

In [16]:
res = collection.query(
  expr = "",
  offset = 60,
  limit = 10, 
  output_fields = ['id', 'embedding']
)

TODO

1) Check embedding size of Lllamaindex Huggingface embettings in (are they ~300 or 2k)
2) See if fields can be inserted to Milvus via Llamaindex. i.e. more than just embeddings



In [17]:
docs[0]

Document(id_='398cc74a-580d-41c2-a85f-0b980e4ff0ef', embedding=None, metadata={'url': 'https://gpt-index.readthedocs.io/en/latest'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='767b9945a76c081d709c631a4c4bd5ea20b3b2ca30c6b52133fa78638aab3be3', text='LlamaIndex is a data framework for LLM-based applications to ingest, structure, and access private or domain-specific data. It’s available in Python (these docs) and Typescript.\n🚀 Why LlamaIndex?\uf0c1\nLLMs offer a natural language interface between humans and data. Widely available models come pre-trained on huge amounts of publicly available data like Wikipedia, mailing lists, textbooks, source code and more.\nHowever, while LLMs are trained on a great deal of data, they are not trained on your data, which may be private or specific to the problem you’re trying to solve. It’s behind APIs, in SQL databases, or trapped in PDFs and slide decks.\nYou may choose to fine-tune a LLM with your data, b

In [18]:
len(docs)

8

In [19]:
docs[7]

Document(id_='23866397-0d5f-4e02-b705-a5e6d425319c', embedding=None, metadata={'url': 'https://docs.llamaindex.ai/en/latest/examples/query_engine/sub_question_query_engine.html'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='08eb1f1778c927a93d31cf7c99d603416ed32583fdce9c16000074dbe00b0a3f', text='Back to top \nToggle table of contents sidebar\nSub Question Query Engine\uf0c1\nIn this tutorial, we showcase how to use a sub question query engine to tackle the problem of answering a complex query using multiple data sources.\nIt first breaks down the complex query into sub questions for each relevant data source, then gather all the intermediate reponses and synthesizes a final response.\nPreparation\uf0c1\nIf you’re opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.\nimport os\nimport openai\n\nos.environ["OPENAI_API_KEY"] = "sk-..."\nopenai.api_key = os.environ["OPENAI_API_KEY"]\n\nimport nest_asyncio\n\nnest_asynci